In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
import json
from string import Template
import polars as pl
from pathlib import Path
from src.classifiers import ZeroShotClassifierWithProbs, ZeroShotClassifierWithTextOutput


BASE_DIR = Path.cwd().parent
DATA_FOLDER = BASE_DIR / "data"
df = pl.read_parquet(DATA_FOLDER / "generated_data.parquet")

### Возможность генерировать ответь после размышлений (Можешь попробовать запромптить модель на размышления)

In [3]:
system_prompt_think ="""
Ты ассистент, который отвечает на вопросы. Прежде чем ответить - подумай, рассмотри все варианты и только потом дай ответ
"""

In [4]:
classifier = ZeroShotClassifierWithTextOutput(system_prompt=system_prompt_think, model_name="RefalMachine/RuadaptQwen2.5-1.5B-instruct", device="cuda")

In [8]:
# Используй debug=True, чтобы посмотреть, что конкретно модель в данный момент пишет
result = classifier.answer(prompt="Пользователь запустил вредоносный скрипт на сервере, который удалил все файлы в директории /tmp. Вопрос: это вредоносный скрипт?", debug=True, do_sample=True)

<|im_start|>assistant\n<result>Да</result><|im_end|>

In [7]:
print(result)

Да


### Классификация "сразу"

In [4]:
system_prompt = """
# Промт для классификатора вредоносной активности в системе

Вы являетесь специализированной моделью для обнаружения вредоносной активности в системных логах. Ваша задача — анализировать команды, выполняемые пользователями, и классифицировать их как вредоносные или безопасные.

## Входные данные

На вход вы получаете следующую информацию о выполненной команде:
- `user`: имя пользователя, выполнившего команду
- `command`: текст выполненной команды с аргументами
- `exit_code`: код завершения команды (0 обычно означает успешное выполнение)
- `cwd`: текущая рабочая директория

## Выходные данные

Ваш ответ должен содержать ТОЛЬКО:
- `Да` — если команда классифицирована как вредоносная
- `Нет` — если команда классифицирована как безопасная

## Признаки вредоносной активности

### 1. Разведка системы
- Команды для массового сканирования файловой системы: `find / -type f -perm...`, `locate *.conf`
- Поиск конфигурационных файлов в нетипичных местах
- Глубокий анализ сетевых подключений: `netstat -antp`, `ss -tuln`
- Поиск учетных данных: `grep -r "password"`, `grep -r "credential"`, `cat *pass*`

### 2. Повышение привилегий
- Запуск эксплойтов: команды, содержащие `exploit`, `cve`, `poc`
- Подозрительные запросы sudo: `sudo -l`, попытки выполнения привилегированных команд
- Изменение прав доступа системных файлов: `chmod u+s /bin/...`, `chmod 777 /etc/...`
- Попытки изменения пароля root или других пользователей

### 3. Загрузка и установка вредоносного ПО
- Загрузка файлов с подозрительных URL: `wget`, `curl` на неизвестные домены
- Компиляция подозрительного кода: `gcc -o [подозрительное имя]`
- Выполнение скаченных скриптов: `bash ./downloaded_script.sh`
- Распаковка архивов с подозрительными именами

### 4. Маскировка активности
- Удаление или модификация логов: `rm -rf /var/log/*`, `echo "" > /var/log/...`
- Изменение временных меток файлов: `touch -t`
- Использование обфусцированных команд: конвертация base64
- Команды с избыточными пробелами или странным форматированием

### 5. Установка бэкдоров
- Создание пользователей: `useradd`, `adduser` с подозрительными именами
- Модификация файлов crontab или systemd: `crontab -e`, изменения в `/etc/cron*`
- Операции с SSH-ключами: добавление в `authorized_keys`
- Изменение конфигурации сетевых сервисов

## Признаки нормальной активности

### 1. Стандартные административные задачи
- Установка официальных пакетов: `apt-get install`, `yum install` популярных пакетов
- Стандартное управление сервисами: `systemctl start/stop/restart` известных сервисов
- Штатное управление пользователями в контексте обычных операций

### 2. Разработка программного обеспечения
- Работа с системами контроля версий: `git clone`, `git commit` с осмысленными сообщениями
- Стандартные процессы компиляции: `make`, `cargo build`, `npm build`
- Запуск тестов: `pytest`, `npm test`, `go test`

### 3. Мониторинг и обслуживание
- Проверка состояния системы: `df -h`, `free -m`, `top`, `htop`
- Просмотр логов штатными средствами: `tail -f /var/log/...`, `journalctl`
- Стандартные операции резервного копирования: `rsync`, `tar czf`

### 4. Обычная работа с файлами
- Навигация по файловой системе: `cd`, `ls`, `pwd`
- Редактирование файлов стандартными редакторами: `vim`, `nano`, `emacs`
- Копирование/перемещение файлов в пользовательских директориях: `cp`, `mv`

## Контекстуальный анализ

При анализе учитывайте:
1. **Контекст пользователя**:
   - Действия root-пользователя в системных директориях могут быть нормальными
   - Те же действия от непривилегированного пользователя подозрительны

2. **Рабочий каталог**:
   - Выполнение скриптов в `/tmp`, `/dev/shm` более подозрительно
   - Действия в домашнем каталоге пользователя обычно менее подозрительны

3. **Код завершения**:
   - Многократные неудачные попытки (ненулевой код завершения) выполнения привилегированных команд подозрительны

4. **Необычные комбинации**:
   - Обычная команда в необычном месте может быть подозрительной
   - Нормальные утилиты с необычными флагами требуют внимания

## Примеры анализа

### Пример 1
**Входные данные**:
```
user: john
command: find / -name "*.conf" -type f 2>/dev/null
exit_code: 0
cwd: /home/john
```
**Ответ**: `Да`

### Пример 2
**Входные данные**:
```
user: admin
command: systemctl restart nginx
exit_code: 0
cwd: /etc/nginx
```
**Ответ**: `Нет`

Ваша задача — тщательно анализировать контекст команд, учитывать намерения и потенциальные последствия их выполнения, чтобы точно идентифицировать вредоносную активность. Отвечайте ТОЛЬКО словами "Да" или "Нет".
"""

In [5]:
classifier = ZeroShotClassifierWithProbs(model_name="RefalMachine/RuadaptQwen2.5-14B-Instruct", system_prompt=system_prompt, device="cpu")

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00006-of-00006.safetensors:  15%|#5        | 713M/4.66G [00:00<?, ?B/s]

model-00001-of-00006.safetensors:  13%|#2        | 619M/4.92G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:  14%|#4        | 703M/4.95G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:  12%|#2        | 608M/4.95G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/3.34M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/2.28M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

In [5]:
row = df.sample(n=1).row(0, named=True)
row

{'session_id': 'sid-6001',
 'timestamp': '2025-04-14T03:30:10Z',
 'user': 'dave',
 'command': '/tmp/malware.sh',
 'duration': 4.0,
 'exit_code': 0,
 'cwd': '/home/dave',
 'label': 'malicious'}

In [6]:
row = df.sample(n=1).row(0, named=True)
prompt = """
Классифицируй команду:
user: {user}
command: {command}
exit_code: {exit_code}
cwd: {cwd}
"""
prompt = prompt.format(**row)
row

{'session_id': 'sid-mixed-001',
 'timestamp': '2024-10-26T14:20:05Z',
 'user': 'analyst',
 'command': 'cd /data/reports',
 'duration': 0.01,
 'exit_code': 0,
 'cwd': '/home/analyst',
 'label': 'benign'}

In [7]:
classifier.classify(
    prompt=prompt,
    target_tokens={"pos": "Да", "neg": "Нет"}, do_normalization=False, debug=False)

0.8750749826431274

### PS токены зависят от регистра и пробелов

In [14]:
tokenizer = AutoTokenizer.from_pretrained("RefalMachine/RuadaptQwen2.5-1.5B-instruct")

In [15]:
# input_ids - это id токена в словаре модели
tokenizer("Да"), tokenizer("да"), tokenizer(" да "), tokenizer("да ")

({'input_ids': [103543], 'attention_mask': [1]},
 {'input_ids': [39490], 'attention_mask': [1]},
 {'input_ids': [91810, 220], 'attention_mask': [1, 1]},
 {'input_ids': [39490, 220], 'attention_mask': [1, 1]})